In [49]:
from sqlalchemy import create_engine, Float, text
from db_connections import con
import datetime
from datetime import timezone, datetime

import pandas as pd


# Execute the SQL query
# result = con.execute("SELECT * FROM my_vocabs;")

# # Fetch all the rows from the query result
# rows = result.fetchall()
# rows = pd.DataFrame(rows, columns=["vocab", "exposure", "status", "img_url1", "img_url2", "img_url3", 
#                                    "created_at_utc", "memorized_at_utc", "confidence", "quizzed_count",
#                                    "correct_count", "incorrect_count", "user_id"])

# split the vocab column by " " and get all the rows with length less than 5
# rows = rows[rows['vocab'].str.split().str.len() < 5]

# If there is "_" in vocab column, replace it with " "
# rows['vocab'] = rows['vocab'].str.replace("_", " ")

# users = con.execute("SELECT * FROM users;")
# users = users.fetchall()
users = pd.DataFrame({"user": "Andy Lee", "user_id": "C02VDLCB52N", "language": "English", "country": "United States", "created_at_utc": datetime.now(timezone.utc), "updated_at_utc": datetime.now(timezone.utc)}, index=[0])

users.to_sql('users', con=con, if_exists='replace', index=False)

1

In [37]:
import difflib
import nltk
from nltk.corpus import wordnet

def is_valid_word(word):
    if wordnet.synsets(word):
        return True
    return False

def find_closest_word(word):
    valid_words = set(wordnet.words())
    closest_match = difflib.get_close_matches(word, valid_words, n=1, cutoff=0.8)
    if closest_match:
        return closest_match[0]
    return None

# Example usage
input_word = input("Enter a word: ")

if is_valid_word(input_word):
    print(f"{input_word} is a valid word.")
else:
    closest_word = find_closest_word(input_word)
    if closest_word:
        print(f"{input_word} is not a valid word. Did you mean {closest_word}?")
    else:
        print(f"{input_word} is not a valid word, and no close match was found.")


love bug is not a valid word, and no close match was found.


In [50]:
import os
import json
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from secret import slack_credentials

slack_token = slack_credentials()
client = WebClient(token=slack_token)

def send_slack_message(channel_id, vocabulary_name):
    try:
        # Construct the Slack message blocks
        blocks = [
            {
                "type": "section",
                "text": {
                    "type": "mrkdwn",
                    "text": f"*Vocabulary:* {vocabulary_name}"
                }
            },
            {
                "type": "actions",
                "elements": [
                    {
                        "type": "button",
                        "text": {
                            "type": "plain_text",
                            "text": "👎"
                        },
                        "value": "confident"
                    },
                    {
                        "type": "button",
                        "text": {
                            "type": "plain_text",
                            "text": "😐"
                        },
                        "value": "neutral"
                    },
                    {
                        "type": "button",
                        "text": {
                            "type": "plain_text",
                            "text": "👍"
                        },
                        "value": "not_confident"
                    }
                ]
            }
        ]

        # Send the Slack message
        response = client.chat_postMessage(
            channel=channel_id,
            blocks=blocks
        )

        # Print the response from Slack API (optional)
        print(response)

    except SlackApiError as e:
        print(e.response)
        print(f"Error sending Slack message: {e.response['error']}")

# Usage: Specify the channel ID and vocabulary name
channel_id = "C02VDLCB52N"  # Replace with your channel ID
vocabulary_name = "Timid"

# Send the Slack message
send_slack_message(channel_id, vocabulary_name)


/Users/andylee/opt/anaconda3/lib/python3.9/site-packages/slack_sdk/web/internal_utils.py:290: UserWarning: The top-level `text` argument is missing in the request payload for a chat.postMessage call - It's a best practice to always provide a `text` argument when posting a message. The `text` argument is used in places where content cannot be rendered such as: system push notifications, assistive technology such as screen readers, etc.
  warnings.warn(missing_text_message, UserWarning)


{'ok': True, 'channel': 'C02VDLCB52N', 'ts': '1684715595.502829', 'message': {'bot_id': 'B02UZ22GFJT', 'type': 'message', 'text': '*Vocabulary:* Timid :-1: button :neutral_face: button :+1: button', 'user': 'U02VDM8LRQT', 'ts': '1684715595.502829', 'app_id': 'A02V70E9P38', 'blocks': [{'type': 'section', 'block_id': 'oJwZy', 'text': {'type': 'mrkdwn', 'text': '*Vocabulary:* Timid', 'verbatim': False}}, {'type': 'actions', 'block_id': 'SBr', 'elements': [{'type': 'button', 'action_id': 'wjJAU', 'text': {'type': 'plain_text', 'text': ':-1:', 'emoji': True}, 'value': 'confident'}, {'type': 'button', 'action_id': 'NEI', 'text': {'type': 'plain_text', 'text': ':neutral_face:', 'emoji': True}, 'value': 'neutral'}, {'type': 'button', 'action_id': 'GSEn1', 'text': {'type': 'plain_text', 'text': ':+1:', 'emoji': True}, 'value': 'not_confident'}]}], 'team': 'T01MB5Z619S', 'bot_profile': {'id': 'B02UZ22GFJT', 'app_id': 'A02V70E9P38', 'name': 'vocab', 'icons': {'image_36': 'https://a.slack-edge.com

In [19]:
import requests

# Make a GET request to the /payloads endpoint
response = requests.get('http://199.241.139.206:8080/payloads')

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Extract the payloads from the response
    payloads = response.json()
    print(payloads)

    # Process the payloads as needed
    for payload in payloads:
        # Extract the necessary data from the payload
        text = payload.get('message', {}).get('text', '')
        button_clicked = payload.get('actions', [{}])[0].get('text', {}).get('text', '')
        timestamp = payload.get('actions', [{}])[0].get('action_ts', '')
        user = payload.get('user', {}).get('username', '')

        # Process the extracted data
        print(f"Text: {text}")
        print(f"Button Clicked: {button_clicked}")
        print(f"Timestamp: {timestamp}")
        print(f"User: {user}")
else:
    # Request was not successful, handle the error
    print(f"Error: {response.status_code}")


[{'actions': [{'action_id': 'zYp', 'action_ts': '1684638324.967756', 'block_id': 'QsX', 'text': {'emoji': True, 'text': ':+1:', 'type': 'plain_text'}, 'type': 'button', 'value': 'not_confident'}], 'api_app_id': 'A02V70E9P38', 'channel': {'id': 'C02VDLCB52N', 'name': 'vocab_practice'}, 'container': {'channel_id': 'C02VDLCB52N', 'is_ephemeral': False, 'message_ts': '1684636205.314459', 'type': 'message'}, 'enterprise': None, 'is_enterprise_install': False, 'message': {'app_id': 'A02V70E9P38', 'blocks': [{'block_id': 'gY=l', 'text': {'text': '*Vocabulary:* Timid', 'type': 'mrkdwn', 'verbatim': False}, 'type': 'section'}, {'block_id': 'QsX', 'elements': [{'action_id': 'tbGn', 'text': {'emoji': True, 'text': ':-1:', 'type': 'plain_text'}, 'type': 'button', 'value': 'confident'}, {'action_id': 'AgMB', 'text': {'emoji': True, 'text': ':neutral_face:', 'type': 'plain_text'}, 'type': 'button', 'value': 'neutral'}, {'action_id': 'zYp', 'text': {'emoji': True, 'text': ':+1:', 'type': 'plain_text'

In [19]:
from secret import lingua_credentials
import requests, json
vocab = 'costa rica'
url = "https://lingua-robot.p.rapidapi.com/language/v1/entries/en/" + \
    vocab.lower().strip(' ')

headers = {
    "X-RapidAPI-Key": lingua_credentials(),
    "X-RapidAPI-Host": "lingua-robot.p.rapidapi.com"
}

# Request Data
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)

def extract_audio_url(json_data):
    
    try:
        json_data = data['entries']
        for entry in json_data:
            pronunciations = entry.get('pronunciations', [])
            for pronunciation in pronunciations:
                audio = pronunciation.get('audio')
                if audio:
                    return audio['url']
    except IndexError:
        return None
    return None

a = extract_audio_url(data)
print(a)

None
